In [1]:
import pandas as pd
import numpy as np

# Mostrar todas las columnas
pd.set_option('display.max_columns', None)

# Mostrar todas las filas (si quieres limitarlo, usa por ejemplo 200)
pd.set_option('display.max_rows', None)

# Ampliar el ancho de cada columna para ver bien el texto
pd.set_option('display.max_colwidth', None)

# Ajustar el ancho de la tabla en consola
pd.set_option('display.width', 0)

In [9]:
import xmlrpc.client
import pandas as pd

# --- Conexión ---
url = "https://donsson.com"
db = "Donsson_produccion"
username = "juan.cano@donsson.com"
password = "1000285668"

common = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/common")
uid = common.authenticate(db, username, password, {})
models = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/object")

# --- 1️⃣ Órdenes de producción terminadas desde 2023 ---
producciones = models.execute_kw(
    db, uid, password,
    "mrp.production", "search_read",
    [[("state", "=", "done"), ("date_finished", ">=", "2023-01-01")]],
    {"fields": ["id", "name", "date_finished", "product_id", "product_qty"]}
)
df_mo = pd.DataFrame(producciones)
print(f"Órdenes de producción encontradas: {len(df_mo)}")

# --- Limpiar producto terminado ---
if "product_id" in df_mo.columns:
    df_mo["producto_terminado_id"] = df_mo["product_id"].apply(lambda x: x[0] if isinstance(x, list) else None)
    df_mo["producto_terminado"] = df_mo["product_id"].apply(lambda x: x[1] if isinstance(x, list) else None)

# --- 2️⃣ Apuntes contables de Materias Primas ---
account_lines = models.execute_kw(
    db, uid, password,
    "account.move.line", "search_read",
    [[("account_id.code", "=", "1405051002"), ("ref", "!=", False)]],
    {"fields": ["ref", "date", "product_id", "quantity", "credit", "debit", "name", "product_uom_id"]}
)
df_acc = pd.DataFrame(account_lines)
print(f"Apuntes contables descargados: {len(df_acc)}")

# --- 3️⃣ Normalizar productos y unidades ---
if "product_id" in df_acc.columns:
    df_acc["product_id_int"] = df_acc["product_id"].apply(lambda x: x[0] if isinstance(x, list) else None)
    df_acc["producto"] = df_acc["product_id"].apply(lambda x: x[1] if isinstance(x, list) else None)
if "product_uom_id" in df_acc.columns:
    df_acc["unidad"] = df_acc["product_uom_id"].apply(lambda x: x[1] if isinstance(x, list) else None)

# --- 4️⃣ Unir con MOs ---
df_acc["MO"] = df_acc["ref"].fillna(df_acc["name"]).astype(str)
df_acc = df_acc[df_acc["MO"].isin(df_mo["name"])]

df_final = df_acc.merge(
    df_mo[["name", "date_finished", "producto_terminado","product_qty"]],
    left_on="MO", right_on="name", how="left"
)

# --- 5️⃣ Calcular costo total ---
df_final["costo_total"] = df_final["credit"].fillna(0) - df_final["debit"].fillna(0)

# --- 6️⃣ Seleccionar columnas limpias ---
df_resultado = df_final[[
    "MO", "date", "producto", "quantity", "unidad",
    "costo_total", "date_finished", "producto_terminado","product_qty"
]]


Órdenes de producción encontradas: 4625
Apuntes contables descargados: 197095


In [11]:
df_resultado.head(10)

,MO,date,producto,quantity,unidad,costo_total,date_finished,producto_terminado,product_qty
0,MO13341,2025-11-13,[ISC] DESMODUR 44V20L (COD FAB),0.22,kg,4039.0,2025-11-13 09:23:18,[DAR03144025] DA3144 FILTRO AIRE 1o. (025 DA3144),20.0
1,MO13341,2025-11-13,[ISC] DESMODUR 44V20L (COD FAB),0.78,kg,14320.0,2025-11-13 09:23:18,[DAR03144025] DA3144 FILTRO AIRE 1o. (025 DA3144),20.0
2,MO13341,2025-11-13,[POL516] SEAFOM 06 B NARANJA (COD FAB),1.31,kg,22434.0,2025-11-13 09:23:18,[DAR03144025] DA3144 FILTRO AIRE 1o. (025 DA3144),20.0
3,MO13341,2025-11-13,[POL516] SEAFOM 06 B NARANJA (COD FAB),1.29,kg,22092.0,2025-11-13 09:23:18,[DAR03144025] DA3144 FILTRO AIRE 1o. (025 DA3144),20.0
4,MO13341,2025-11-13,[CLORUR] CLORURO DE METILENO (COD FAB),0.90,kg,3870.0,2025-11-13 09:23:18,[DAR03144025] DA3144 FILTRO AIRE 1o. (025 DA3144),20.0
5,MO13341,2025-11-13,[ISCIA1] ISOCIANATO BLANDO (COD FAB),0.07,kg,1274.0,2025-11-13 09:23:18,[DAR03144025] DA3144 FILTRO AIRE 1o. (025 DA3144),20.0
6,MO13341,2025-11-13,[POL60B] SEALFOAM 60 B NEGRO (COD FAB),0.12,kg,2174.0,2025-11-13 09:23:18,[DAR03144025] DA3144 FILTRO AIRE 1o. (025 DA3144),20.0
7,MO13370,2025-11-13,[HMG 850.6] HOTMELT 850.6 JOWAT (COD FAB),0.67,kg,15408.0,2025-11-13 09:26:36,[DAB08234025] DA8234 FILTRO AIRE - DONGFENG (025 DA8234),200.0
8,MO13370,2025-11-13,[HMG 850.6] HOTMELT 850.6 JOWAT (COD FAB),6.23,kg,143273.0,2025-11-13 09:26:36,[DAB08234025] DA8234 FILTRO AIRE - DONGFENG (025 DA8234),200.0
9,MO13371,2025-11-13,[HMG 850.6] HOTMELT 850.6 JOWAT (COD FAB),3.40,kg,78191.0,2025-11-13 09:27:41,"[DAR02190025] DA2190 FILTRO AIRE 1_ ATLAS COPCO VOLVO. Ver carcasa GXC221, GXKL15 (025 DA2190)",100.0
